In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

#  options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [64]:
jupyter trust notebook-name.ipynb

# Webscrape the Toronto Neighborhoods

In [2]:
#  list of postal codes of Toronto area contains all the information we need to assign:
#  
#      PostalCode, Borough, Neighborhood
#
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url)
soup = BeautifulSoup(html_data.text, 'html.parser')

In [3]:
#  how many tables does the webpage contain?
print('There are {} tables defined in the webpage'.format(len(soup.find_all('table'))))

There are 3 tables defined in the webpage


In [4]:
#  pick the table with first cell entry: M1A
table = [tbl for tbl in soup.find_all('table') if 'M1A' in tbl.find('tbody').find('tr').find('td').text]
table = table[0].find('tbody')

Scrape the table and make sure to identify multiple neighborhoods and comma-separate them. It seems by the code example given from the instructor, namely:

(((((cell.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')

that cases like M3K with an entry like North York (Downsview) East (CFB Toronto) should have the Neighborhood defined only as Downsview East, i.e. any extra parenthesized entry like (CFB Toronto) should be ignored.

In [5]:
toronto = pd.DataFrame(columns=['PostalCode', 'Borough', 'Neighborhood'])
for cell in table.find_all('td'):
    if cell.span.text == 'Not assigned':
        continue
    #  capture postal code, borough and all neighborhoods
    postalcode, borough, neighborhoods = re.search(r'(M\d[A-Z])([^(]+)\((.*)$', cell.text.strip()).groups()
    
    #  remove all extra parenthesized neighborhoods
    #  replace any leftover closed parenthesis with space
    #  replace any / with comma
    neighborhoods = neighborhoods.split('(')[0].strip(')').replace(')', ' ').replace(' /', ',').strip(' ')
    toronto.loc[len(toronto)] = [postalcode, borough, neighborhoods]

In [6]:
print(f'Total number of neighborhoods found = {len(toronto)}')

Total number of neighborhoods found = 103


In [7]:
#Identify Boroughs with unusually long names. We might have to manually fix them.
toronto.loc[ toronto.Borough.str.len() > 20 ]

,PostalCode,Borough,Neighborhood
35,M4J,East YorkEast Toronto,The Danforth East
76,M7R,MississaugaCanada Post Gateway Processing Centre,Enclave of L4W
92,M5W,Downtown TorontoStn A PO Boxes25 The Esplanade,Enclave of M5E
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L


In [8]:
tofix = toronto.loc[ toronto.Borough.str.len() > 20, 'PostalCode']
toronto.loc[ toronto.Borough.str.len() > 20, 'Borough' ] = ['East York',
                                                            'Mississauga',
                                                            'Downtown Toronto',
                                                            'East Toronto']
toronto[ toronto.PostalCode.isin(tofix) ]

,PostalCode,Borough,Neighborhood
35,M4J,East York,The Danforth East
76,M7R,Mississauga,Enclave of L4W
92,M5W,Downtown Toronto,Enclave of M5E
100,M7Y,East Toronto,Enclave of M4L


In [9]:
#  are there any Boroughs with 'Not assigned' neighborhoods?
toronto.query('Neighborhood.str.contains("[Nn]ot").values')

,PostalCode,Borough,Neighborhood


In [10]:
toronto.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [11]:
toronto.shape

(103, 3)

# Geocode the Toronto Neighborhoods

In [12]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [13]:
#  get the Toronto latitude and longitude coordinates
gl = Nominatim(user_agent='toronto_neighborhoods')
toronto_gl = gl.geocode('Toronto, Ontario')
toronto_lat , toronto_lng = toronto_gl.latitude, toronto_gl.longitude
print(f'Toronto latitude = {toronto_lat} and longitude = {toronto_lng}')

Toronto latitude = 43.6534817 and longitude = -79.3839347


In [14]:
#  let's get the rest of the Toronto Neighborhood coordinates if we can get it from Google Maps
def assign_latlng(postalcode):
    coords = None
    STOP = 10
    while (coords is None):
        g = geocoder.google(f'{postalcode}, Toronto, Ontario')
        coords = g.latlng
        STOP -= 1
        if (STOP == 0):
            print('It does not work..')
            break
    return coords

In [30]:
# Import downloaded CSV file and use it

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_ccf4b5bb060649ed946e45d78d1b6db6 = 'https://s3.eu-geo.objectstorage.softlayer.net'
else:
    endpoint_ccf4b5bb060649ed946e45d78d1b6db6 = 'https://s3.eu-geo.objectstorage.service.networklayer.com'

client_ccf4b5bb060649ed946e45d78d1b6db6 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='vyL0GYvtfdoCoXilukLGS7GR-ZNcMNf-aS-1mzQr7LZJ',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url=endpoint_ccf4b5bb060649ed946e45d78d1b6db6)

body = client_ccf4b5bb060649ed946e45d78d1b6db6.get_object(Bucket='finalcapstoneproject-donotdelete-pr-zaf3a6iginul7u',Key='Geospatial_Coordinates.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

toronto_latlng = pd.read_csv(body)

toronto_latlng.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)

#  there should not be any differences in Postal Codes between our webscrapped set and this one
assert( set(toronto_latlng.PostalCode) == set(toronto.PostalCode) )
toronto_latlng.head()


,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
#  inner join
toronto = toronto.merge(toronto_latlng, on='PostalCode')

In [31]:
toronto.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [32]:
toronto.shape

(103, 5)

# Explore the Toronto Neighborhoods

Let's create first a map of all the neighborhoods.



In [35]:
!pip install folium

     |████████████████████████████████| 94 kB 6.3 MB/s  eta 0:00:01


In [37]:
import folium

In [38]:
#  shift location a bit so all neighborhoods fit at zoom level 11
toronto_map = folium.Map(location=[toronto_lat+0.06, toronto_lng+0.05], zoom_start=11)
for lat, lng, label in zip(toronto.Latitude, toronto.Longitude, toronto.Neighborhood):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)
toronto_map

In [39]:
#  prepare to talk to the Foursquare API
CLIENT_ID = 'W3SH5A1VQH330RLC4U5J4XRRC3FYJYMBYDN4KIXV5CFBZNV5'
CLIENT_SECRET = 'XMHABSF1WWR0TZ2AJNM0POZ0FYSU0DDE4YWPS5DMVEHLRQUX'
VERSION = '20180605' # Foursquare API version
LIMIT = 100

In [40]:

#  borrow the wrapper function from the exercise lab
#  modify it when a given neighborhood request does not produce any results not to fail but keep going 
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        try:
            results = requests.get(url).json()["response"]['groups'][0]['items']
            venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
            
        except:
            venues_list.append([(
            name, 
            lat, 
            lng, 
            None, 
            None, 
            None,  
            None)])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [41]:
#  query Foursquare for top 100 venues in each of the Toronto Neighborhoods
toronto_venues = getNearbyVenues(toronto.Neighborhood, toronto.Latitude, toronto.Longitude, radius=500)

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [42]:
# Check if we have unfilled entries.
toronto_venues.applymap(lambda x: x is None).sum()

Neighborhood              0
Neighborhood Latitude     0
Neighborhood Longitude    0
Venue                     0
Venue Latitude            0
Venue Longitude           0
Venue Category            0
dtype: int64

In [48]:
# Let's order neighborhoods by the number of venues returned (up to the maximum limit we chose).
toronto_venues.groupby('Neighborhood')[['Venue']].count().sort_values('Venue', ascending=False)

,Venue
Neighborhood,
"Harbourfront East, Union Station, Toronto Islands",100
"Garden District, Ryerson",100
"First Canadian Place, Underground city",100
Enclave of M5E,100
"Toronto Dominion Centre, Design Exchange",100
"Commerce Court, Victoria Hotel",100
"Richmond, Adelaide, King",94
St. James Town,82
Church and Wellesley,71


In [51]:
# Let's one-hot encode the venue categories in order to start preparing for neighborhood clustering. First let's check what are the different venues.
toronto_venues['Venue Category'].value_counts()

Coffee Shop                        184
Café                                96
Restaurant                          62
Park                                56
Pizza Place                         46
Italian Restaurant                  45
Bakery                              43
Hotel                               42
Japanese Restaurant                 41
Sandwich Place                      40
Gym                                 34
Clothing Store                      32
Fast Food Restaurant                31
Sushi Restaurant                    31
Bank                                28
Bar                                 27
Grocery Store                       26
Pub                                 25
Seafood Restaurant                  24
Thai Restaurant                     23
Pharmacy                            23
Breakfast Spot                      22
American Restaurant                 21
Gastropub                           18
Bookstore                           18
Beer Bar                 

In [52]:
# As it turns out there is a Neighborhood venue category in there!
toronto_venues.query('`Venue Category` == "Neighborhood"')

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
347,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
586,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood
769,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood
1244,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood


In [55]:
# This is going to mess up the downstream analysis, where we already rely on the Neighborhood column for grouping, so let's replace these values with Neighboorhood Area.
toronto_venues['Venue Category'].replace({'Neighborhood': 'Neighborhood Area'}, inplace=True)
toronto_venues[ toronto_venues['Venue Category'].str.contains('Neigh') ]

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
347,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood Area
586,"Richmond, Adelaide, King",43.650571,-79.384568,Downtown Toronto,43.653232,-79.385296,Neighborhood Area
769,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,Harbourfront,43.639526,-79.380688,Neighborhood Area
1244,Studio District,43.659526,-79.340923,Leslieville,43.662070,-79.337856,Neighborhood Area


In [56]:
features = pd.get_dummies(toronto_venues[['Venue Category']], prefix='', prefix_sep='')
features = pd.concat([toronto_venues[['Neighborhood']], features], axis=1)
features.head()

,Neighborhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [57]:
# Let's compute the venue frequencies for each neighborhood and then identify the top 10 venues per neighborhood.
features_freqs = features.groupby('Neighborhood').mean().reset_index()

In [58]:
def getTop10(row):
    #  get the neighborhood venue frequencies as a single row
    #  order them in descending order
    #  take the top10
    p = row.iloc[0].sort_values(ascending=False).iloc[:10].reset_index().set_axis(['top venue', 'freq'], axis=1)
    
    #  make sure zero frequency venues do not count in the top10 list
    p.loc[ p.freq.eq(0.0), 'top venue' ] = None
    
    #  rename the index as Top1, Top2, ..., Top10
    p['name'] = ['Top' + str(i) for i in range(1, 11)]
    p.set_index('name', inplace=True)
    
    return p['top venue']

#  ok, let's run this function now neighborhood by neighborhood on our DataFrame
top10 = features_freqs.groupby('Neighborhood').apply(getTop10)
top10.head()

name,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10
Neighborhood,,,,,,,,,,
Agincourt,Skating Rink,Lounge,Breakfast Spot,Latin American Restaurant,None,None,None,None,None,None
"Alderwood, Long Branch",Pizza Place,Sandwich Place,Gym,Coffee Shop,Skating Rink,Pub,None,None,None,None
"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Fried Chicken Joint,Supermarket,Sushi Restaurant,Mobile Phone Shop,Deli / Bodega,Bridal Shop,Frozen Yogurt Shop,Middle Eastern Restaurant
Bayview Village,Japanese Restaurant,Café,Bank,Chinese Restaurant,None,None,None,None,None,None
"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Sandwich Place,Greek Restaurant,Butcher,Sushi Restaurant,Juice Bar,Fast Food Restaurant,Liquor Store,Café


# Cluster the Toronto Neighborhoods

In [59]:
from sklearn.cluster import KMeans

In [60]:
# Let's try to cluster neighborhoods into four major clusters. For than we need to summarize the one-hot encoding of each neighborhood into a sparse one-hot version where each neighborhood has a single row of one-hot values.

n_clusters = 3
features_sparse = features.groupby('Neighborhood').sum().reset_index()
kmeans = KMeans(n_clusters=n_clusters, random_state=42).fit(features_sparse.drop('Neighborhood', axis=1))
features_sparse.insert(1, column='cluster', value=kmeans.labels_)
features_sparse.head()

,Neighborhood,cluster,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [61]:
features_sparse.cluster.value_counts()

1    77
2    17
0     6
Name: cluster, dtype: int64

In [62]:
# The cluster representation looks relatively ok, let's create an annotated map of neighborhoods!

toronto_clustered = toronto.loc[ toronto.Neighborhood.isin(features_sparse.Neighborhood) ]\
.set_index('Neighborhood').join(features_sparse.set_index('Neighborhood'), on='Neighborhood').reset_index()
toronto_clustered.head()


,Neighborhood,PostalCode,Borough,Latitude,Longitude,cluster,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bike Shop,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Auditorium,College Gym,College Rec Center,College Stadium,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Curling Ice,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant,Donut Shop,Drugstore,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hospital,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kids Store,Korean BBQ Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Luggage Store,Market,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Motel,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood Area,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,River,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,South American Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,M3A,North York,43.753259,-79.329656,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [63]:
toronto_map = folium.Map(location=[toronto_lat, toronto_lng], zoom_start=11)

cluster_color = {0: 'red', 1: 'steelblue', 2: 'green'}

for lat, lng, neighborhood, cluster in zip(toronto_clustered.Latitude, 
                                           toronto_clustered.Longitude, 
                                           toronto_clustered.Neighborhood,
                                           toronto_clustered.cluster):
    label = folium.Popup(str(neighborhood) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=cluster_color[cluster],
        fill=True,
        fill_color=cluster_color[cluster],
        fill_opacity=1.0,
        parse_html=False).add_to(toronto_map)
toronto_map